# Using a pre-defined analyzer

This example will show you how to use one of the pre-configured analyzers provided by sfgad.

For this, we will create an example dataset, select one of the pre-defined analyzers, train it on several iterations of the dataset, evaluate it on the last iteration and use an aggregative step to extract the most anomalous subgraph from it.

### Generate Example Dataset

Before we select and train our analyzer, we need an exemplary dataset. For this, we use one of the generators that come with the framework.

We generate a graphstream of $T = 30$ iterations with each iteration featuring the same $n = 100$ vertices. Further, the generator inserts $m = 500$ edges by randomly connecting vertices. Note, that this model does not allow loops or multiedges and generates an undirected graph.

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import datetime as dt

In [2]:
def generate_graph(n_vertices, n_edges, n_vertex_types):
    graph = nx.gnm_random_graph(n_vertices, n_edges)
    for v, data in graph.nodes(data=True):
        data['type'] = v % n_vertex_types
    for u, v, data in graph.edges(data=True):
        data['type'] = str(graph.node[u]['type']) + '_' + str(graph.node[v]['type'])
    return graph

In [3]:
DF_COLUMNS = ['TIMESTAMP', 'E_NAME', 'E_TYPE', 'SRC_NAME', 'SRC_TYPE', 'DST_NAME', 'DST_TYPE']

def from_nx(network, timestep=0):
    df = np.asarray([[dt.datetime.fromordinal(1).replace(year=2017) + dt.timedelta(days=timestep),
                      data.get('name', str((i, j))),
                      data.get('type', 'E_TYPE'),
                      str(i),
                      network.node[i].get('type', 'V_TYPE'),
                      str(j),
                      network.node[j].get('type', 'V_TYPE')] for (i, j, data) in network.edges(data=True)])
    return pd.DataFrame(data=df, columns=DF_COLUMNS)

In [4]:
def create_dataset(n_vertices, n_edges, n_vertex_types, n_timesteps):         
    return [from_nx(generate_graph(n_vertices, n_edges, n_vertex_types), timestep=i) for i in range(n_timesteps)]

In [5]:
graph_stream = create_dataset(n_vertices=100, n_edges=500, n_vertex_types=3, n_timesteps=30)

### Initialize Analyzer

To use a pre-configured analyzer, we need to import the algorithms module.

This module provides several existing graph-based anomaly detection approaches. For our example, we will use the DAPA-V10 algorithm. It relies on the vertex degree as a feature and assumes a gaussian distribution for estimate the probability of the current state.

In [6]:
from sfgad import algorithms

analyzer = algorithms.dapa(n_jobs=1)

### Analyze Graphstream

In order to gain significant insight about the probability of the current state of a vertex, we need a sufficient amount of obsverations as comparison.

Since we are interested in analyzing the last iteration of the graphstream, we fit the analyzer on all but the last iteration. 

In [7]:
for g in graph_stream[:-1]:
    analyzer.fit_transform(g)

Following, we evaluate the last iteration using the fitted analyzer.

In [8]:
result = analyzer.fit_transform(graph_stream[-1])

The results show p-values ranging between 0 and 1.

In [9]:
result.sort_values(by='p_value')

,name,time_window,p_value
17,17,29,0.031243
41,41,29,0.041766
62,62,29,0.053797
87,88,29,0.063193
55,55,29,0.074539
28,28,29,0.078837
36,36,29,0.096292
12,12,29,0.100281
1,1,29,0.102407
26,26,29,0.121813


Further, we can see that the p-values roughly follow a uniform dsitribution, as we would expect.

In [10]:
result['p_value'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

count    100.000000
mean       0.480085
std        0.299781
min        0.031243
10%        0.127279
20%        0.195624
30%        0.254904
40%        0.333833
50%        0.395100
60%        0.551872
70%        0.666600
80%        0.805902
90%        0.934373
max        0.999370
Name: p_value, dtype: float64

### Extract Most Anomalous Subgraph

To extract the most anomalous (connected) subgraph, we use approaches from the aggregation module. These provide the vertices that are part of the anomaly and a score indicating its degree of anomalousness.

For our example, we use the NPHGS graph-scan with a significance threshold $\alpha_{max} = 0.2$ and $K = 5$ seed vertices.

In [11]:
from sfgad.aggregation import graph_scan

detected_vertices, score = graph_scan.scan(graph_stream[-1], result, alpha_max=0.2, K=5)

Linking the extracted vertices back to its p-values shows that are all below the singificance threshold $\alpha_{max}$.

Further, we can see that some vertices with lower p-values are not included in the extracted subgraph. This stems from the fact that they are not connected through edges to the subgraph. Vertices with higher p-values are not included since they would decrease the score of the anomaly.

In [12]:
result[result['name'].isin(detected_vertices)].sort_values('p_value')

,name,time_window,p_value
55,55,29,0.074539
12,12,29,0.100281
